In [116]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.ensemble
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors as desc
from rdkit.DataStructs import cDataStructs as ds
from rdkit.DataStructs import UIntSparseIntVect as usv
from rdkit.Chem import MACCSkeys, DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem.AtomPairs import Pairs
import csv

In [117]:
from rdkit.Chem.rdchem import Mol

In [118]:
from rdkit.Chem.rdchem import RingInfo

In [2]:
import rdkit.Chem.Fingerprints
fulltrain = pd.read_csv("train (1) 2.csv")
fulltest = pd.read_csv("test (1).csv")

In [188]:
tmorgs = []
for smile in strain[20000:20100]:
    m = Chem.MolFromSmiles(smile)
    morgan = AllChem.GetMorganFingerprint(m,2)
    tmorgs.append(morgan)

In [189]:
tmorgs

In [121]:
ytrain = fulltrain['gap']
strain = fulltrain['smiles']
stest = fulltest['smiles']

In [150]:
stest[1]

'[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1'

In [151]:
cols = ['m','ringInfo','brackets','num_heavy','num_atoms','num_bonds','num_rings','num_bond_rings','num_atom_rings','bond_rings','atom_rings','bonds','aroms']

In [ ]:
fopen = open("train_engineer.csv", 'wb')
transform_csv = csv.writer(fopen, delimiter=',', quotechar='"')
fopen = open("train_engineer_sims.csv", 'wb')
sims_csv = csv.writer(fopen, delimiter=',', quotechar='"')


features = pd.DataFrame([range(len(cols))],columns=cols)
sim_df = pd.DataFrame([range(5)],columns=None)
i = 0
for smile in strain:
    m = Chem.MolFromSmiles(smile)
    ringInfo = Mol.GetRingInfo(m)
    num_heavy = Mol.GetNumHeavyAtoms(m)
    num_atoms = Mol.GetNumAtoms(m)
    num_bonds = Mol.GetNumBonds(m)
    num_rings = RingInfo.NumRings(ringInfo)
    num_bond_rings = RingInfo.NumBondRings(ringInfo,2)
    num_atom_rings = RingInfo.NumAtomRings(ringInfo,2)
    bond_rings = RingInfo.BondRings(ringInfo)
    atom_rings = RingInfo.AtomRings(ringInfo)
    bonds = Mol.GetBonds(m)
    aroms = Mol.GetAromaticAtoms(m)
    morg = AllChem.GetMorganFingerprint(m,2)
    sims = []
    for tmorg in tmorgs[:5]:
        sim = DataStructs.DiceSimilarity(morg,tmorg)
        sims.append(sim)
    count = 0
    for char in smile:
        if char in ["[","]"]:
            count = count+1
    brackets = count
    f1 = [m,ringInfo,brackets,num_heavy,num_atoms,num_bonds,num_rings,num_bond_rings,num_atom_rings,bond_rings,atom_rings,bonds,aroms]
    sim_df.loc[i] = sims
    features.loc[i] = f1
    i = i+1
    if i%5000==0:
        print i
    transform_csv.writerow(f1)
    sims_csv.writerow(sims)

In [ ]:
fopen = open("test_sims_all.csv", 'wb')
transform_csv = csv.writer(fopen, delimiter=',', quotechar='"')
test_sims = pd.DataFrame()
prog = 0

for smile in test_smiles: 
    smile_sim = []
    prog+= 1
    if prog%10000==0:
        print prog
    m = Chem.MolFromSmiles(smile)
    fp = AllChem.GetMorganFingerprintAsBitVect(m,2,useFeatures=True)
    for comp in comp_morgans:
        sim = DataStructs.DiceSimilarity(comp,fp)
        smile_sim.append(sim)
    transform_csv.writerow(smile_sim)

In [187]:
sim_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [201]:
features_train = features[['brackets','num_heavy','num_atoms','num_rings','num_bond_rings','num_atom_rings','bond_rings','atom_rings']]
features_train.join(sim_df)

,brackets,num_heavy,num_atoms,num_rings,num_bond_rings,num_atom_rings,bond_rings,atom_rings,0,1,...,90,91,92,93,94,95,96,97,98,99
0,2,27,27,6,1,1,"((0, 1, 2, 3, 26), (5, 27, 8, 7, 6), (11, 12, ...","((0, 1, 2, 3, 4), (6, 5, 9, 8, 7), (11, 12, 13...",0.310345,0.444444,...,0.564972,0.514620,0.517241,0.367816,0.333333,0.283688,0.515723,0.456790,0.474359,0.366013
1,4,25,25,6,1,1,"((0, 1, 2, 3, 24), (6, 26, 19, 27, 25, 5), (8,...","((0, 1, 2, 3, 4), (6, 7, 19, 20, 24, 5), (8, 9...",0.476190,0.307692,...,0.397661,0.339394,0.380952,0.309524,0.376812,0.562963,0.366013,0.307692,0.346667,0.353741
2,4,27,27,7,1,2,"((0, 1, 20, 28, 26), (2, 1, 27, 29, 3), (5, 14...","((0, 1, 2, 21, 26), (3, 2, 1, 20, 4), (5, 6, 1...",0.379310,0.481481,...,0.531073,0.514620,0.471264,0.505747,0.347222,0.382979,0.377358,0.481481,0.410256,0.575163
3,6,27,27,7,1,2,"((0, 29, 8, 9, 26), (2, 31, 5, 4, 3), (6, 31, ...","((0, 1, 8, 9, 10), (3, 2, 6, 5, 4), (7, 6, 2, ...",0.413793,0.432099,...,0.350282,0.374269,0.321839,0.367816,0.347222,0.297872,0.352201,0.370370,0.333333,0.287582
4,0,29,29,7,1,1,"((28, 27, 29, 2, 1, 0), (4, 30, 20, 32, 5), (7...","((0, 28, 27, 3, 2, 1), (5, 4, 21, 20, 6), (7, ...",0.300000,0.369048,...,0.459016,0.474576,0.444444,0.322222,0.280000,0.258503,0.278788,0.511905,0.518519,0.515723
5,4,29,29,7,1,2,"((0, 1, 29, 27, 28), (3, 4, 30, 26, 29, 2), (6...","((0, 1, 2, 27, 28), (3, 4, 5, 26, 27, 2), (6, ...",0.622222,0.547619,...,0.524590,0.372881,0.422222,0.522222,0.413333,0.394558,0.436364,0.369048,0.358025,0.289308
6,0,22,22,5,1,1,"((0, 1, 2, 3, 21), (6, 7, 8, 9, 22, 5), (10, 8...","((0, 1, 2, 3, 4), (6, 7, 8, 9, 10, 5), (11, 9,...",0.314465,0.489796,...,0.469136,0.358974,0.352201,0.465409,0.465116,0.317460,0.375000,0.380952,0.539007,0.289855
7,2,28,28,7,0,1,"((0, 1, 22, 30, 27), (4, 5, 6, 7, 28, 3), (8, ...","((0, 1, 2, 23, 27), (4, 5, 6, 7, 8, 3), (9, 7,...",0.406780,0.581818,...,0.444444,0.563218,0.384181,0.474576,0.408163,0.305556,0.419753,0.436364,0.440252,0.269231
8,2,25,25,6,1,1,"((0, 1, 2, 3, 24), (5, 25, 23, 26, 6), (8, 27,...","((0, 1, 2, 3, 4), (6, 5, 24, 23, 7), (8, 9, 21...",0.452381,0.410256,...,0.327485,0.315152,0.273810,0.380952,0.420290,0.266667,0.431373,0.346154,0.386667,0.231293
9,4,24,24,6,1,1,"((0, 1, 2, 24, 23), (4, 25, 7, 6, 5), (10, 16,...","((0, 1, 2, 3, 23), (5, 4, 8, 7, 6), (10, 11, 1...",0.533333,0.405229,...,0.309524,0.456790,0.303030,0.351515,0.385185,0.333333,0.306667,0.352941,0.367347,0.291667


In [203]:
reg_train = features_train.drop(['bond_rings','atom_rings'],1)

In [204]:
reg_t = reg_train[:9999]
reg_v = reg_train[10000:20000]

In [167]:
features_train1 = features[['brackets','num_heavy','num_atoms','num_rings','num_bond_rings','num_atom_rings']]

In [210]:
trains = features_train.loc[:9999]
valid = features_train.loc[10000:20000]

In [207]:
lr = sklearn.linear_model.LinearRegression()
lr.fit(reg_t,ytrain[:9999])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [209]:
pred_lr = lr.predict(reg_v)
mean_squared_error(ytrain[10000:20000],pred_lr)

0.12639383875789548

In [ ]:
import os
os.system('say "your program has finished"')

In [165]:
for i in range(features_train.shape[0]):
    x = len(features_train.loc[i])
    if x != len(features_train.loc[1]):
        print "Row ", i, ' - ',x

In [173]:
trains_feat = trains[['brackets','num_heavy','num_atoms','num_rings','num_bond_rings','num_atom_rings']]
valid_feat = valid[['brackets','num_heavy','num_atoms','num_rings','num_bond_rings','num_atom_rings']]

In [213]:
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=100)
rf.fit(reg_t,ytrain[:9999])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [215]:
preds = rf.predict(reg_v)
mean_squared_error(ytrain[10000:20000],preds)

0.11249518158400185

In [72]:
names = Mol.GetPropNames(Chem.MolFromSmiles(strain[1]))
print names
# arr = np.zeros((1,))
# DataStructs.ConvertToNumpyArray(names, arr)
# arr

In [231]:
cols

['m',
 'ringInfo',
 'brackets',
 'num_heavy',
 'num_atoms',
 'num_bonds',
 'num_rings',
 'num_bond_rings',
 'num_atom_rings',
 'bond_rings',
 'atom_rings',
 'bonds',
 'aroms']

In [36]:
morgans = []
for smile in strain[:200]:
    m = Chem.MolFromSmiles(smile)
    morgan = AllChem.GetMorganFingerprint(m,2,invariants=[1]*m.GetNumAtoms(),useBondTypes=False)
    morgans.append(morgan)
    

In [37]:
tmorgans = []
for smile in strain[10000:16000]:
    m = Chem.MolFromSmiles(smile)
    morgan = AllChem.GetMorganFingerprint(m,2,invariants=[1]*m.GetNumAtoms(),useBondTypes=False)
    tmorgans.append(morgan)

In [38]:
vmorgans = []
for smile in strain[20000:26000]:
    m = Chem.MolFromSmiles(smile)
    morgan = AllChem.GetMorganFingerprint(m,2,invariants=[1]*m.GetNumAtoms(),useBondTypes=False)
    vmorgans.append(morgan)

In [39]:
sims = pd.DataFrame()
for morg in morgans:
    comp_sim = []
    for vmorg in vmorgans:
        sim = DataStructs.DiceSimilarity(morg,vmorg)
        comp_sim.append(sim)
    sims[comp] = comp_sim

NameError: name 'comp' is not defined

In [ ]:
vsims = pd.DataFrame()
for morg in morgans:
    comp_sim = []
    for vmorg in tmorgans:
        sim = DataStructs.DiceSimilarity(morg,vmorg)
        comp_sim.append(sim)
    vsims[morg] = comp_sim

In [229]:
train_engineer = pd.read_csv("train_engineer.csv",nrows=80000,header=None)

In [233]:
m_morg = Chem.MolFromSmiles(strain[1])
morgan = AllChem.GetMorganFingerprint(m_morg,2)

In [236]:
sims = []
for smile in strain:
    m = Chem.MolFromSmiles(smile)
    morg = AllChem.GetMorganFingerprint(m,2)
    sims.append(DataStructs.DiceSimilarity(morg,morgan))

KeyboardInterrupt: 

In [250]:
sims_df = pd.DataFrame(sims[:train_engineer_feats.shape[0]])
sims_df

,0
0,0.320513
1,1.000000
2,0.371795
3,0.461538
4,0.333333
5,0.444444
6,0.368794
7,0.364780
8,0.306667
9,0.394558


In [252]:
#train_engineer_feats
tf = train_engineer_feats.join(sims_df)

In [253]:
xtrain = fulltrain.drop(['smiles','gap'],1)

In [257]:
tra = xtrain.loc[:80000]

In [269]:
ttt = tra.join(tf)
tt = ttt.drop([9,10],1)
tt = tt.drop(80000)

In [248]:
train_engineer_feats = train_engineer.drop([0,1,11,12],1)

In [318]:
split = 60000
X_train = tt.loc[:split]
y_train = ytrain[:split+1]
X_test = tt.loc[split:]
y_test = ytrain[split+1:split+1+20000]
y_test

60001    1.23
60002    2.00
60003    1.86
60004    1.97
60005    1.88
60006    2.07
60007    1.80
60008    1.92
60009    1.45
60010    1.51
60011    1.25
60012    2.33
60013    1.89
60014    1.95
60015    2.14
60016    1.89
60017    2.47
60018    1.87
60019    1.82
60020    1.53
60021    1.30
60022    1.39
60023    1.94
60024    1.23
60025    1.27
60026    1.87
60027    1.87
60028    1.84
60029    2.11
60030    1.90
         ... 
79971    1.86
79972    2.03
79973    1.07
79974    1.79
79975    2.23
79976    2.08
79977    1.95
79978    2.17
79979    2.06
79980    1.78
79981    1.19
79982    2.09
79983    1.84
79984    2.41
79985    2.11
79986    2.40
79987    1.05
79988    2.20
79989    2.83
79990    1.82
79991    2.08
79992    1.89
79993    1.52
79994    1.45
79995    1.60
79996    1.67
79997    2.05
79998    2.47
79999    2.35
80000    1.36
Name: gap, dtype: float64

In [308]:
print(__doc__)

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

Automatically created module for IPython interactive environment


In [ ]:
from sklearn.linear_model import Lasso

lasso_errors = []
enet_errors = []
i = 0
while i <= 2:
    alpha = i
    lasso = Lasso(alpha=alpha)
    y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
    lasso_errors.append(mean_squared_error(y_test,y_pred_lasso))
    

In [333]:
enet_errors_11rat = []
i = 0
while i <= 2:  
    enet = ElasticNet(alpha=alpha, l1_ratio=i)
    y_pred_enet = enet.fit(X_train, y_train).predict(X_test)
    enet_errors_11rat.append(mean_squared_error(y_test,y_pred_enet))
    i = i+0.1
    
print enet_errors_11rat

[0.2030691162094016, 0.1826431975139432, 0.17349900289348644, 0.16834570640569199, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773]


In [331]:
print lasso_errors
print enet_errors

[0.2654755101714299, 0.19344025148447436, 0.18506432275131932, 0.1788161564380267, 0.17389075711166671, 0.1702881247722392, 0.16800825941974429, 0.16705116105418194, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773]
[0.2654755101714299, 0.19703308371324621, 0.18932215597630656, 0.18418411631580506, 0.17976707729044022, 0.17601923558205954, 0.1729328741401242, 0.17050036549998446, 0.16871417062707303, 0.16756683777785603, 0.1670510013772703, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773, 0.16704341104451773]


In [327]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet(alpha=alpha, l1_ratio=0.7)

y_pred_enet = enet.fit(X_train, y_train).predict(X_test)
r2_score_enet = r2_score(y_test, y_pred_enet)
#print(enet)
#print("r^2 on test data : %f" % r2_score_enet)

# plt.plot(enet.coef_, color='lightgreen', linewidth=2,
#          label='Elastic net coefficients')
# plt.plot(lasso.coef_, color='gold', linewidth=2,
#          label='Lasso coefficients')
# plt.plot(coef, '--', color='navy', label='original coefficients')
# plt.legend(loc='best')
# plt.title("Lasso R^2: %f, Elastic Net R^2: %f"
#           % (r2_score_lasso, r2_score_enet))
# plt.show()

ElasticNet(alpha=0.8, copy_X=True, fit_intercept=True, l1_ratio=0.7,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)
r^2 on test data : -0.010005
0.168714170627


In [ ]:
enet_08 = mean_squared_error(y_test,y_pred_enet)
lasso_08 = mean_squared_error(y_test,y_pred_lasso)
print enet

In [337]:
enet = ElasticNet(alpha=1.3, l1_ratio=1)
preds_enet = enet.fit(tt, ytrain[:80000]).predict(ts)

In [ ]:
lasso = sklearn.linear_model.Lasso(alpha=1.3)
lasso.fit(tt,ytrain[:80000])
preds_sub1 = lasso.predict(ts)
preds_sub = pd.DataFrame(testId)

In [340]:
bayrid = sklearn.linear_model.BayesianRidge()
bayrid.fit(tt,ytrain[:80000])
preds_bayrid = bayrid.predict(ts)
bayrid_sub = pd.DataFrame(testId)

In [338]:
bayrid_sub['Prediction'] = preds_bayrid

In [353]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(tt[:60000],ytrain[:60000])
dtest = xgb.DMatrix(tt[60000:])
# specify parameters via map
param = {'max_depth':11, 'eta':1, 'silent':1, 'objective':'reg:linear' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)
mean_squared_error(preds,ytrain[60000:80000])

0.046782262074518133

In [368]:
tt.columns=[range(tt.shape[1])]
ts.columns=[range(ts.shape[1])]
ts

,0,1,2,3,4,5,6,7,8,9,...,254,255,256,257,258,259,260,261,262,263
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2,29,29,35,7,0,1,0.358024691358
2,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4,23,23,28,6,1,1,0.291666666667
3,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,6,31,31,38,8,1,1,0.404761904762
4,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,4,28,28,34,7,1,1,0.465408805031
5,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2,32,32,39,8,0,1,0.25730994152
6,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2,21,21,25,5,1,2,0.391304347826
7,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,6,27,27,33,7,1,1,0.371794871795
8,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2,29,29,35,7,1,1,0.543209876543
9,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2,32,32,38,7,0,1,0.304093567251
10,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,6,27,27,33,7,1,1,0.371794871795


In [369]:
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(tt,ytrain[:80000])
dtest = xgb.DMatrix(tss)
# specify parameters via map
param = {'max_depth':11, 'eta':1, 'silent':1, 'objective':'reg:linear' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)
#mean_squared_error(preds,ytrain[60000:80000])

In [377]:

pred125 = pd.DataFrame(testId)
pred125["Prediction"] = preds

In [380]:
pred125.to_csv('1.25am_predictions.csv',index=None)

In [305]:
print error_01
print error_04
print error_06
print error_10
print error_13

0.193435039103
0.173880789753
0.167994852103
0.167028243318
0.167028243318


In [271]:
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=100)
rf.fit(tt,ytrain[:80000])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [275]:
testId = fulltest['Id']

In [279]:
ts = teststuff.drop(0)

In [280]:
preds = pd.DataFrame(testId)
results = rf.predict(ts)
preds['Prediction'] = results

#mean_squared_error(yvalid,preds)

In [282]:
preds.to_csv("finalsubs.csv",index=None)

In [283]:
preds

,Id,Prediction
0,1,1.865183
1,2,2.047050
2,3,1.472950
3,4,1.485156
4,5,1.557446
5,6,2.439030
6,7,1.940895
7,8,1.815500
8,9,1.845705
9,10,2.239234


In [274]:
teststuff = pd.read_csv('testparams.csv',header=None)

/Users/hikarisorensen/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,2

In [273]:
teststuff

NameError: name 'teststuff' is not defined

In [281]:
os.system('say "your program has finished"')

0